In [ ]:
import sys
import os
sys.path.append(os.pardir)

import rl
from mlagents_envs.environment import UnityEnvironment
import numpy as np

rl.util.seed(0)

In [ ]:
try:
    env.close()
except:
    pass

env = UnityEnvironment(file_name="../ML-Agents Environments/Windy Gridworld")
env.reset()

In [ ]:
behavior_name = list(env.behavior_specs)[0]
print(f"Name of the behavior: {behavior_name}")

In [ ]:
spec = env.behavior_specs[behavior_name]

print(f"Number of observation: {len(spec.observation_specs)}")

In [ ]:
print(f"There are {spec.action_spec.discrete_size} discrete action branches")
if spec.action_spec.discrete_size > 0:
    for i, action_count in enumerate(spec.action_spec.discrete_branches):
        print(f"Branch {i} has {action_count} actions")